This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](catmull-rom.ipynb)

# Derivation of Uniform Catmull-Rom Splines

tangent vectors:

\begin{equation}
\boldsymbol{\dot{x}}_i = \frac{\boldsymbol{x}_{i+1} - \boldsymbol{x}_{i-1}}{2}
\end{equation}

In [ ]:
import sympy as sp
sp.init_printing()

In [ ]:
from utility import NamedExpression, NamedMatrix

In [ ]:
from helper import plot_basis

To keep the indices simple,
we are considering the fifth spline segment
from $\boldsymbol{x}_4$ to $\boldsymbol{x}_5$,
but the results can of course easily be generalized
to an arbitrary segment
from $\boldsymbol{x}_i$ to $\boldsymbol{x}_{i+1}$.

Reminder: [Hermite spline](hermite-uniform.ipynb) segments
use the start and end positions as well as the tangent vectors
at start and end:

In [ ]:
control_values_H = sp.Matrix(sp.symbols('xbm4:6 xdotbm4:6'))
control_values_H

Catmull-Rom splines use 4 positions instead:
The start and end positions of the current segment ($\boldsymbol{x}_4$ and $\boldsymbol{x}_5$) plus the start position of the previous segment ($\boldsymbol{x}_3$) and the end position of the following segment ($\boldsymbol{x}_6$).

TODO: figure? more explanations ...

In [ ]:
x3, x4, x5, x6 = sp.symbols('xbm3:7')

In [ ]:
control_values_CR = sp.Matrix([x3, x4, x5, x6])
control_values_CR

In [ ]:
xd4 = NamedExpression('xdotbm4', (x5 - x3) / 2)
xd4

In [ ]:
xd5 = NamedExpression('xdotbm5', (x6 - x4) / 2)
xd5

So let's look for a way to transform Catmull-Rom control values to Hermite control values.
Since we already have $M_\text{H}$ from [the notebook about uniform Hermite splines](hermite-uniform.ipynb), we can use it to get $M_\text{CR}$:

In [ ]:
M_H = NamedMatrix(
    r'{M_\text{H}}',
    sp.Matrix([[2, -2, 1, 1], [-3, 3, -2, -1], [0, 0, 1, 0], [1, 0, 0, 0]]))
M_H

In [ ]:
M_CRtoH = NamedMatrix(r'{M_{\text{CR$\to$H}}}', 4, 4)

In [ ]:
M_CR = NamedMatrix(r'{M_\text{CR}}', M_H.name * M_CRtoH.name)
M_CR

In [ ]:
NamedMatrix(control_values_H, M_CRtoH.name * control_values_CR)

If we substitute the above definitions of
$\boldsymbol{\dot{x}}_4$ and
$\boldsymbol{\dot{x}}_5$,
we can directly read off the matrix elements:

In [ ]:
M_CRtoH.expr = sp.Matrix([
    [expr.coeff(cv) for cv in control_values_CR]
    for expr in control_values_H.subs([xd4.args, xd5.args])])
M_CRtoH.pull_out(sp.S.Half)

In [ ]:
print(_.expr)

In [ ]:
M_HtoCR = NamedMatrix(r'{M_{\text{H$\to$CR}}}', M_CRtoH.I.expr)
M_HtoCR

In [ ]:
print(_.expr)

In [ ]:
M_CR = M_CR.subs_symbols(M_H, M_CRtoH).doit()
M_CR.pull_out(sp.S.Half)

In [ ]:
print(_.expr)

And for completeness' sake, its inverse:

In [ ]:
M_CR.I

In [ ]:
print(_.expr)

In [ ]:
t = sp.symbols('t')

In [ ]:
b_CR = NamedMatrix(
    r'{b_\text{CR}}',
    sp.Matrix([t**3, t**2, t, 1]).T * M_CR.expr)
b_CR.T.pull_out(sp.S.Half)

In [ ]:
plot_basis(
    *b_CR.expr,
    labels=sp.symbols('xbm_i-1 xbm_i xbm_i+1 xbm_i+2'))